Deep Learning
=============

Assignment 5 | Tianzi Cai | 2016-07-24

------------

The goal of this assignment is to train a Word2Vec skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [2]:
try:
  import tensorflow as tf
  print("TensorFlow is already installed")
except ImportError:
  print("Installing TensorFlow")
  import subprocess
  subprocess.check_call(["/databricks/python/bin/pip", "install", "https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-0.6.0-cp27-none-linux_x86_64.whl"])
  print("TensorFlow has been installed on this cluster")

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
# %matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
# from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

Download the data from the source website if necessary.

In [5]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
    '''Return a tuple (filename, headers) where filename is the local file name under which the object can be found, 
    and headers is whatever the info() method of the object returned by urlopen() returned (for a remote object, possibly cached). 
    Exceptions are the same as for urlopen().'''
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

In [6]:
urlretrieve(url + filename, filename)

In [7]:
os.stat(filename)

Read the data into a string.

In [9]:
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

In [10]:
with zipfile.ZipFile(filename) as f:
  print(f.namelist()) # namelist() Return a list of archive members by name
  print(f.namelist()[0])
  print(len(f.namelist()))
  print(type(f.read(f.namelist()[0])))
  print(len(f.read(f.namelist()[0])))
  print(type(tf.compat.as_str(f.read(f.namelist()[0]))))
  print(len(tf.compat.as_str(f.read(f.namelist()[0]))))

In [11]:
print(type(words))
print(words[:10])

Build the dictionary and replace rare words with UNK token.

In [13]:
print(collections.Counter(['tianzi', 'tianzi', 'ronny']))
print(collections.Counter(['tianzi', 'tianzi', 'ronny', 'ronny', 'y']).most_common(2))
x = [['UNK', -1]]
x.extend(collections.Counter(['tianzi', 'tianzi', 'ronny']).most_common(2))
print(x)
x.append(collections.Counter(['t', 't', 'r']).most_common(2))
x # append() and extend() are destructive operations

In [14]:
vocabulary_size = 50000
count = [['UNK', -1]]
count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
dictionary = dict()
for word, _ in count:
  dictionary[word] = len(dictionary) # way to add index, which is like the rank, except for 'UNK'
  
print(count[:3])
print(dictionary['UNK'])
print(dictionary['the'])
print(len(dictionary))

In [15]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1)) #https://docs.python.org/dev/library/collections.html#collections.Counter
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
# del words  # Hint to reduce memory.

In [16]:
{k: dictionary[k] for k in dictionary.keys()[:2]}

Function to generate a training batch for the skip-gram model.

In [18]:
# print(words[:4])
# {k: dictionary[k] for k in words[:4]}

In [19]:
x = data[:7]
xbuffer = collections.deque(maxlen = 5)
'''the deque is bounded to the specified maximum length. Once a bounded length deque is full, when new items are added, a corresponding number of items are discarded from the opposite end'''
x_index=0
print (x)
for _ in range(8):
  xbuffer.append(x[x_index])
  x_index = (x_index + 1) % len(x)
  print(xbuffer)

In [20]:
data_index = 0
num_skips = 2
skip_window = 1
batch_size = 8
                           
batch = np.ndarray(shape=(batch_size), dtype=np.int32) # 1D array
labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32) # 2D array
span = 2 * skip_window + 1 # [ skip_window, target, skip_window ]
buffer = collections.deque(maxlen=span) # cool! NEVER used before

for _ in range(span):
  buffer.append(data[data_index])
  data_index = (data_index + 1) % len(data)

print (buffer)

for i in range(batch_size // num_skips):
  target = skip_window
  targets_to_avoid = [ skip_window ]
  print (targets_to_avoid)
  for j in range(num_skips): 
    print('j is %d'%j)
    while target in targets_to_avoid: 
      target = random.randint(0, span - 1) 
    targets_to_avoid.append(target)
    print(targets_to_avoid)
    batch[i * num_skips + j] = buffer[skip_window] 
    print(buffer[skip_window])
    labels[i * num_skips + j, 0] = buffer[target]
    print(buffer[target])
    
  buffer.append(data[data_index])
  print(buffer)
  data_index = (data_index + 1) % len(data)
  #print(data_index)
  print('\n')
#return batch, labels

In [21]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0 # makes sure that batch_size is divisible by num_skips
  assert num_skips <= 2 * skip_window # pick x out of 2n possible words surrounding target word, x<=2n
  batch = np.ndarray(shape=(batch_size), dtype=np.int32) # 1D array
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32) # 2D array
  span = 2 * skip_window + 1 # [ skip_window, target, skip_window ]
  buffer = collections.deque(maxlen=span) # cool! NEVER used before
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
    
  for i in range(batch_size // num_skips): # 8//2 = 4
    target = skip_window  # first target is right after the first skip_window
    targets_to_avoid = [ skip_window ] # [1]
    for j in range(num_skips): # 2
      while target in targets_to_avoid: # [1]
        target = random.randint(0, span - 1) # choose from 0 to 3-1
      targets_to_avoid.append(target) # [1,2]
      batch[i * num_skips + j] = buffer[skip_window] # buffer[1]
      labels[i * num_skips + j, 0] = buffer[target] # buffer[2]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window) # qhy batch_size = 8?
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

In [22]:
# okay, this finally makes sense to me. 
print('data:', [reverse_dictionary[di] for di in data[:16]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=16, num_skips=num_skips, skip_window=skip_window) # batch_size = 8?
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(16)])

In [23]:
# okay, this finally makes sense to me. The model is randomly sampling words surrounding it
print('data:', [reverse_dictionary[di] for di in data[:16]])

for num_skips, skip_window in [(2, 1), (3, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=12, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(12)])

In [24]:
batch_size = 128 #  16 <= batch_size <= 512 # https://www.tensorflow.org/versions/r0.7/tutorials/word2vec/index.html#vector-representations-of-words
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input (target) to generate a label. (word)
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. (Yes)
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size)) #huh
num_sampled = 64 # Number of negative examples to sample. #huh

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size]) # 128
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1]) # 128, 1
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32) # 16
  
  # Variables. minval=0, maxval=None
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], minval = -1.0, maxval = 1.0)) # "input" word vector/embeddings/weight between the input word and the hidden layer
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size))) # 50000, 128
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size])) # 50000
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset) # this acts like indexing lookup because train contains indices
  '''http://stackoverflow.com/questions/34870614/what-does-tf-nn-embedding-lookup-function-do'''
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights = softmax_weights, biases = softmax_biases, inputs = embed,
                               labels = train_labels, num_sampled = num_sampled, num_classes = vocabulary_size))
  '''(tz)
  tf.nn.sampled_softmax_loss() is different from tf.nn.softmax() because sometimes there is a huge number of classes,
  like this the example above, which has essentially 50000 classes. tf.nn.sampled_softmax_loss() is a faster way to train
  a softmax classifier over a huge number of classes. it returns the sampled softmax training loss. it is generally an 
  underestimate of the full softmax loss. it is for training only. 
  '''

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed. (tz: wow! cool!)
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True)) # 50000, 128
  normalized_embeddings = embeddings / norm # 50000, 128
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset) # 16, 128
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings)) # 16 by 50000

In [25]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

In [26]:
print(batch_data)
print(batch_labels)

In [27]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [28]:
import pandas as pd
df = (sqlContext.createDataFrame(pd.DataFrame(two_d_embeddings))) # 400
display(df)

In [29]:
valid_examples

In [30]:
final_embeddings

In [31]:
print(similarity.get_shape())
x = similarity[3, :]
print(x)

An alternative to skip-gram is another Word2Vec model called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. Implement and evaluate a CBOW model trained on the text8 dataset.

In [33]:
'''https://gist.github.com/discorev/b6a0900a52b62cd04f33'''

import numpy as np

data_index = 0

def generate_batch_cbow(batch_size, skip_window):
  global data_index
  context_window = 2 * skip_window # calculate the context_window - this is the total number of words around the target
  assert batch_size % context_window == 0 # ensure the context window can be taken from the batch size
  num_labels = batch_size / context_window # the number of labels is the how many context windows fit in the batch
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(num_labels, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(num_labels):
    target = skip_window  # target label at the center of the buffer
    labels[i, 0] = buffer[target] # set the label
    targets_to_avoid = [ skip_window ]
    for j in range(context_window):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * context_window + j] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

skip_window = 1
batch, labels = generate_batch_cbow(8, skip_window)
print(np.shape(labels))
for i in range(8):
  print(batch[i], '->', labels[i/(2*skip_window), 0])
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i/(2*skip_window), 0]])
del skip_window # remove skip_window setting used for testing

In [34]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(xrange(valid_window), valid_size))
num_sampled = 32 # Number of negative examples to sample.

## General defines
context_window = 2 * skip_window
num_labels = batch_size / context_window

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[num_labels, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)

  # seq_ids only needs to be generated once so do this as a numpy array rather than a tensor.
  seq_ids = np.zeros(batch_size, dtype=np.int32)
  cur_id = -1
  for i in range(batch_size):
    if i % context_window == 0:
      cur_id = cur_id + 1
    seq_ids[i] = cur_id
  print(seq_ids)
  
  # use segment_sum to add together the related words and reduce the output to be num_labels in size.
  final_embed = tf.segment_sum(embed, seq_ids)
  '''https://www.tensorflow.org/versions/master/api_docs/python/math_ops.html#segment_sum'''
  
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, final_embed,
                               train_labels, num_sampled, vocabulary_size))

  # Optimizer.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [35]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  average_loss = 0
  for step in xrange(num_steps):
    batch_data, batch_labels = generate_batch_cbow(batch_size, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step", step, ":", average_loss)
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = "%s %s," % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

In [36]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(xrange(valid_window), valid_size))
num_sampled = 32 # Number of negative examples to sample.

## General defines
context_window = 2 * skip_window
num_labels = batch_size / context_window

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[num_labels, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)

  # seq_ids only needs to be generated once so do this as a numpy array rather than a tensor.
  seq_ids = np.zeros(batch_size, dtype=np.int32)
  cur_id = -1
  for i in range(batch_size):
    if i % context_window == 0:
      cur_id = cur_id + 1
    seq_ids[i] = cur_id
  print(seq_ids)
  
  # use segment_mean to add together the related words and reduce the output to be num_labels in size.
  final_embed = tf.segment_mean(embed, seq_ids)
  '''https://www.tensorflow.org/versions/master/api_docs/python/math_ops.html#segment_sum'''
  
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, final_embed,
                               train_labels, num_sampled, vocabulary_size))

  # Optimizer.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [37]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  average_loss = 0
  for step in xrange(num_steps):
    batch_data, batch_labels = generate_batch_cbow(batch_size, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step", step, ":", average_loss)
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = "%s %s," % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()